In [2]:
# import our libraries
import requests
heads = {'Host': 'www.sec.gov', 'Connection': 'close',
         'Accept': 'application/json, text/javascript, */*; q=0.01', 'X-Requested-With': 'XMLHttpRequest',
         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36',
         }
import urllib
from bs4 import BeautifulSoup

In [3]:
# let's first make a function that will make the process of building a url easy.
def make_url(base_url , comp):
    
    url = base_url
    
    # add each component to the base url
    for r in comp:
        url = '{}/{}'.format(url, r)
        
    return url

# EXAMPLE
base_url = r"https://www.sec.gov/Archives/edgar/data"
components = ['886982','000156459019011378', '0001564590-19-011378-index-headers.html']
make_url(base_url, components)

'https://www.sec.gov/Archives/edgar/data/886982/000156459019011378/0001564590-19-011378-index-headers.html'

In [4]:
# define the urls needed to make the request, let's start with all the daily filings
base_url = r"https://www.sec.gov/Archives/edgar/daily-index"

# The daily-index filings, require a year and content type (html, json, or xml).
year_url = make_url(base_url, ['2020', 'index.json'])

# Display the new Year URL
print('-'*100)
print('Building the URL for Year: {}'.format('2020'))
print("URL Link: " + year_url)

# request the content for 2019, remember that a JSON strucutre will be sent back so we need to decode it.
content = requests.get(year_url, headers= heads)
print(content)
decoded_content = content.json()

# the structure is almost identical to other json requests we've made. Go to the item list.
# AGAIN ONLY GRABBING A SUBSET OF THE FULL DATASET 
file_urls= []
for item in decoded_content['directory']['item'][0:4]:
    
    # get the name of the folder
    print('-'*100)
    print('Pulling url for Quarter: {}'.format(item['name']))
    
    # The daily-index filings, require a year, a quarter and a content type (html, json, or xml).
    qtr_url = make_url(base_url, ['2020', item['name'], 'index.json'])
    
    # print out the url.
    print("URL Link: " + qtr_url)
    
    # Request, the new url and again it will be a JSON structure.
    file_content = requests.get(qtr_url, headers= heads)
    decoded_content = file_content.json()
    
    print('-'*100)
    print('Pulling files')

    # for each file in the directory items list, print the file type and file href.
    # AGAIN DOING A SUBSET
    for file in decoded_content['directory']['item']:
        if 'master' in file['name']:
            file_urls.append(make_url(base_url, ['2020', item['name'], file['name']]))

----------------------------------------------------------------------------------------------------
Building the URL for Year: 2020
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2020/index.json
<Response [200]>
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR1
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/index.json
----------------------------------------------------------------------------------------------------
Pulling files
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR2
URL Link: https://www.sec.gov/Archives/edgar/daily-index/2020/QTR2/index.json
----------------------------------------------------------------------------------------------------
Pulling files
----------------------------------------------------------------------------------------------------
Pulling url for Quarter: QTR

In [5]:
file_urls

['https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200102.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200103.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200106.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200107.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200108.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200109.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200110.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200113.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200114.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200115.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200116.idx',
 'https://www.sec.gov/Archives/edgar/daily-index/2020/QTR1/master.20200117.idx',
 'https://www.sec.gov/Archiv

In [6]:
master_data_link= []
for file_url in file_urls:
    content = requests.get(file_url, headers= heads).content

    #print(file_url)
    # Now that we loaded the data, we have a byte stream that needs to be decoded and then split by double spaces.
    data = content.decode("utf-8").split('  ')
    #print(data)

    # We need to remove the headers, so look for the end of the header and grab it's index
    for index, item in enumerate(data):
        if "ftp://ftp.sec.gov/edgar/" in item:
            start_ind = index

    # define a new dataset with out the header info.
    data_format = data[start_ind + 1:]

    

    # now we need to break the data into sections, this way we can move to the final step of getting each row value.
    for index, item in enumerate(data_format):

        # if it's the first index, it won't be even so treat it differently
        if index == 0:
            clean_item_data = item.replace('\n','|').split('|')
            clean_item_data = clean_item_data[8:]
        else:
            clean_item_data = item.replace('\n','|').split('|')

        for index, row in enumerate(clean_item_data):

            # when you find the text file.
            if '.txt' in row:

                # grab the values that belong to that row. It's 4 values before and one after.
                mini_list = clean_item_data[(index - 4): index + 1]

                if len(mini_list) != 0:
                    if mini_list[2] == '4':
                        mini_list[4] = "https://www.sec.gov/Archives/" + mini_list[4]
                        document_dict = {}
                        document_dict['reporter_cik_number'] = mini_list[0]
                        document_dict['reporting_person'] = mini_list[1]
                        document_dict['form_id'] = mini_list[2]
                        document_dict['date'] = mini_list[3]
                        document_dict['file_url'] = mini_list[4]
                        
                        master_data_link.append(document_dict)

    # grab the first three items
    if len(master_data_link) != 0:
        print('-'*100)
        print('Date', master_data_link[0]['date'])
        print(master_data_link[:3])

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

...

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/0001437749-20-000076.txt'}, {'reporter_cik_number': '1233126', 'reporting_person': 'SORAN PHILIP', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233126/0001230245-20-000006.txt'}, {'reporter_cik_number': '1233152', 'reporting_person': 'CALHOUN DAVID L', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1233152/0000018230-20-000008.txt'}]
----------------------------------------------------------------------------------------------------
Date 20200102
[{'reporter_cik_number': '1232602', 'reporting_person': 'TRAUB KENNETH H', 'form_id': '4', 'date': '20200102', 'file_url': 'https://www.sec.gov/Archives/edgar/data/1232602/000

In [7]:
len(master_data_link)

176302

In [8]:
# import our libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [13]:
insider_data= pd.DataFrame(columns= ["fid", "cik", "rptownercik", "rptownername", "relationship", "date", "transactioncode", "transactionshares", "transactionpricepershare", "sharesownedfollowingtransaction"])

In [14]:
# define the base url needed to create the file url.
#https://www.sec.gov/Archives/edgar/data/1679788/000167978821000072.txt
import time
base_url = r"https://www.sec.gov"

# convert a normal url to a document url
#normal_url = r"https://www.sec.gov/Archives/edgar/data/1237472/0001725526-19-000007.txt"
for i, form in enumerate(master_data_link):
    time.sleep(0.03)
    normal_url= form['file_url']
    normal_url = normal_url.replace('-','').replace('.txt','/index.json')

    # request the url and decode it.
    print(i, ":", normal_url)
    try:
        content = requests.get(normal_url, headers= heads).json()
    except:
        print("error", "*"*95)
        continue
    for file in content['directory']['item']:
        # Grab the filing summary and create a new url leading to the file so we can download it.
        if '.xml' in file['name']:
            xml_summary = base_url + content['directory']['name'] + "/" + file['name']
    try:
        content = requests.get(xml_summary, headers= heads).content
    except:
        continue
    soup = BeautifulSoup(content, 'lxml')
    data= {}
    if soup.nonderivativetable != None:
        try:
            data['fid']= xml_summary.split('/')[-2]
            data['cik']= soup.issuer.issuercik.text
            data['rptownercik']= soup.reportingowner.reportingownerid.rptownercik.text
            data['rptownername']= soup.reportingowner.reportingownerid.rptownername.text
        except:
            continue
        relationship= soup.reportingowner.reportingownerrelationship
        if relationship.isdirector != None and relationship.isdirector.text == "1":
            data['relationship']= 'Director'
        elif relationship.isofficer != None and relationship.isofficer.text == "1":
            data['relationship']= relationship.officertitle.text.split('&')
        elif relationship.istenpercentowner != None and relationship.istenpercentowner.text == "1":
            data['relationship']= 'Ten Percent Owner'
        elif relationship.isother != None and relationship.isother.text == "1":
            data['relationship']= 'Other'
        else:
            data['relationship']= '-1'

        if soup.nonderivativetable != None:
            for i in soup.nonderivativetable.find_all('nonderivativetransaction'):

                try:
                    data['date']= i.transactiondate.value.text
                except:
                    data['date']= "-1"
                try:
                    data['transactioncode']= i.transactioncoding.transactioncode.text
                except:
                    data['transactioncode']= "-1"
                try:
                    data['transactionshares']= i.transactionamounts.transactionshares.value.text
                except:
                    data['transactionshares']= "-1"
                try:
                    data['transactionpricepershare']= i.transactionpricepershare.value.text
                except:
                    data['transactionpricepershare']= "-1"
                try:
                    data['sharesownedfollowingtransaction']= i.sharesownedfollowingtransaction.value.text
                except:
                    data['sharesownedfollowingtransaction']= "-1"
                insider_data= insider_data.append(data, ignore_index= True)
                print("-"*100)
print(insider_data)
insider_data.to_csv("2020_2.csv")

90000 : https://www.sec.gov/Archives/edgar/data/732712/000161159320000128/index.json
90001 : https://www.sec.gov/Archives/edgar/data/732712/000161159320000129/index.json
90002 : https://www.sec.gov/Archives/edgar/data/732712/000161159320000130/index.json
90003 : https://www.sec.gov/Archives/edgar/data/732712/000161159320000131/index.json
90004 : https://www.sec.gov/Archives/edgar/data/732712/000161159320000132/index.json
90005 : https://www.sec.gov/Archives/edgar/data/732712/000161159320000133/index.json
90006 : https://www.sec.gov/Archives/edgar/data/732712/000161159320000134/index.json
90007 : https://www.sec.gov/Archives/edgar/data/732717/000112760220015209/index.json
90008 : https://www.sec.gov/Archives/edgar/data/732717/000112760220015211/index.json
----------------------------------------------------------------------------------------------------
90009 : https://www.sec.gov/Archives/edgar/data/732717/000112760220015212/index.json
90010 : https://www.sec.gov/Archives/edgar/data/7

----------------------------------------------------------------------------------------------------
90047 : https://www.sec.gov/Archives/edgar/data/7623/000091419020000163/index.json
----------------------------------------------------------------------------------------------------
90048 : https://www.sec.gov/Archives/edgar/data/7623/000091419020000164/index.json
----------------------------------------------------------------------------------------------------
90049 : https://www.sec.gov/Archives/edgar/data/7623/000091419020000165/index.json
----------------------------------------------------------------------------------------------------
90050 : https://www.sec.gov/Archives/edgar/data/763901/000120919120026786/index.json
----------------------------------------------------------------------------------------------------
90051 : https://www.sec.gov/Archives/edgar/data/763901/000120919120026798/index.json
----------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
90091 : https://www.sec.gov/Archives/edgar/data/803164/000090572920000097/index.json
----------------------------------------------------------------------------------------------------
90092 : https://www.sec.gov/Archives/edgar/data/803164/000090572920000098/index.json
----------------------------------------------------------------------------------------------------
90093 : https://www.se

----------------------------------------------------------------------------------------------------
90128 : https://www.sec.gov/Archives/edgar/data/850209/000124015120000031/index.json
----------------------------------------------------------------------------------------------------
90129 : https://www.sec.gov/Archives/edgar/data/850209/000124015120000032/index.json
----------------------------------------------------------------------------------------------------
90130 : https://www.sec.gov/Archives/edgar/data/850209/000124015120000033/index.json
----------------------------------------------------------------------------------------------------
90131 : https://www.sec.gov/Archives/edgar/data/853816/000112760220015181/index.json
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
90132 : https://www.sec.gov/Archives/edgar/data/853816

----------------------------------------------------------------------------------------------------
90159 : https://www.sec.gov/Archives/edgar/data/870385/000087038520000020/index.json
----------------------------------------------------------------------------------------------------
90160 : https://www.sec.gov/Archives/edgar/data/870385/000087038520000021/index.json
----------------------------------------------------------------------------------------------------
90161 : https://www.sec.gov/Archives/edgar/data/872589/000170813820000046/index.json
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
--------------------------------------

...

----------------------------------------------------------------------------------------------------
99879 : https://www.sec.gov/Archives/edgar/data/60519/000120919120030471/index.json
----------------------------------------------------------------------------------------------------
99880 : https://www.sec.gov/Archives/edgar/data/60519/000120919120030566/index.json
----------------------------------------------------------------------------------------------------
99881 : https://www.sec.gov/Archives/edgar/data/60519/000120919120030589/index.json
99882 : https://www.sec.gov/Archives/edgar/data/60519/000120919120030636/index.json
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
99883 : https://www.sec.gov/Archives/edgar/data/60714/0001

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------
99957 : https://www.sec.gov/Archives/edgar/data/742112/000120919120030505/index.json
----------------------------------------------------------------------------------------------------
99958 : https://www.sec.gov/Archives/edgar/data/742112/000120919120030509/index.json
----------------------------------------------------------------------------------------------------
99959 : https://www.sec.gov/Archives/edgar/data/743316/000120919120030641/index.json
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
99960 : https://www.sec.gov/Archives/edgar/data/743316/000120919120030642/index.json
----------------------------------------------------------------------------------------------------
------------------------------------------------------

----------------------------------------------------------------------------------------------------
99996 : https://www.sec.gov/Archives/edgar/data/800457/000120919120030444/index.json
----------------------------------------------------------------------------------------------------
99997 : https://www.sec.gov/Archives/edgar/data/800457/000120919120030457/index.json
----------------------------------------------------------------------------------------------------
99998 : https://www.sec.gov/Archives/edgar/data/802481/000080248120000052/index.json
----------------------------------------------------------------------------------------------------
99999 : https://www.sec.gov/Archives/edgar/data/811156/000124636020001237/index.json
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
------------------------------------------------------